# CNN AutoEncoder Results

Helpful for Time-Series Anamoly Detection 

Articles:

[Anamoly Detection with AutoEncoder](https://towardsdatascience.com/anomaly-detection-with-autoencoder-b4cdce4866a6)

[Machine Learning for Anamoly Detection and Condition Monitoring](https://towardsdatascience.com/how-to-use-machine-learning-for-anomaly-detection-and-condition-monitoring-6742f82900d7)

### Libraries

In [ ]:
%%capture
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score, precision_recall_curve, accuracy_score
from sklearn.metrics import classification_report, roc_curve, PrecisionRecallDisplay, RocCurveDisplay, auc
from sklearn.impute import SimpleImputer
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import AUC
from keras.models import Sequential

import os
import glob
import fileinput
import matplotlib.pyplot as plt
import seaborn as sns

# Import autoencoder for TSC anamoly detection
!pip install pyod
from pyod.models.auto_encoder import AutoEncoder

# MTAKDD-19

In [ ]:
# MTAKDD-19

dfLEG = pd.read_csv('https://raw.githubusercontent.com/IvanLetteri/MTA-KDD-19/master/datasetLegitimate33featues.csv')
dfMTA = pd.read_csv('https://raw.githubusercontent.com/IvanLetteri/MTA-KDD-19/master/datasetMalware33featues.csv')
df = pd.concat([dfMTA, dfLEG])
print(df.shape)
df.head()

(64554, 34)


,FinFlagDist,SynFlagDist,RstFlagDist,PshFlagDist,AckFlagDist,DNSoverIP,TCPoverIP,UDPoverIP,MaxLen,MinLen,...,NumPorts,FlowLEN,FlowLENrx,repeated_pkts_ratio,NumCon,NumIPdst,Start_flow,DeltaTimeFlow,HTTPpkts,label
0,0.478168,1.144246,-0.716937,0.671470,1.089129,-0.154516,0.154711,-0.156991,-0.009901,-1.028609,...,-0.889691,0.479515,0.284428,0.862578,5.253913,5.142702,0.562400,-0.052248,1.487787,1.0
1,-1.204652,0.316746,-0.716937,1.537622,1.135609,-0.154516,0.154711,-0.156991,0.722669,0.628640,...,0.293799,1.301840,-0.036629,-0.908981,-0.190341,-0.194497,0.562400,-0.315281,1.786307,1.0
2,-1.204652,-0.216194,-0.716937,-0.457715,-0.848148,-0.154516,0.154711,-0.156991,-0.172436,0.628640,...,-0.889691,-0.452523,-0.049479,-0.671509,-0.190341,-0.194497,0.562400,-1.197897,0.583908,1.0
3,0.478168,0.636365,-0.716937,1.615480,1.274407,-0.154516,0.154711,-0.156991,0.722669,0.628640,...,0.823293,1.416370,-0.042111,-1.626711,-0.190341,-0.194497,0.562400,-0.097754,1.805515,1.0
4,1.747840,1.867679,1.867097,0.903129,1.473227,6.778562,-7.182148,6.836627,0.485384,0.628640,...,1.116241,0.868445,0.678529,0.902818,5.253913,5.142781,0.563213,0.885878,1.614688,1.0


In [ ]:
# empty list and DataFrame to capture statistics
results = [] 
precision_recall_all = pd.DataFrame()

# split into input (X) and output (y) variables
X = df.iloc[:,:33]
y = df.iloc[:,33]

# split into training (80%), validation (10%), and test (10%) sets
rand_seed = 123
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size = 0.8,
                                                  random_state = rand_seed,
                                                  stratify = y)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, 
                                                    train_size = 0.5,
                                                    random_state = rand_seed,
                                                    stratify = y_rem)

#Creating and fitting model
model = AutoEncoder(contamination=0.05, hidden_neurons = [3,3],
                        epochs = 8, optimizer='Nadam')
model.fit(X_train)
    
# make predictions with model (.50 threshold)
predictions = (model.predict(X_test) > 0.5).astype(int)

# calculate metrics for each model
precision, recall, fscore, support = score(y_test, predictions, 
                                           average = 'binary')
auc = roc_auc_score(y_test, predictions)
accuracy = accuracy_score(y_test, predictions)
    
# append metrics to results
results.append(
    {
      'Accuracy': accuracy,
      'Precision': precision,
      'Recall': recall,
      'F1 Score': fscore,
      'AUC': auc
    })
    
# write data for precision recall curves to csv
cols = ['precision', 'recall', 'thresholds']
precision_recall = precision_recall_curve(y_test, model.predict(X_test))
precision_recall = pd.DataFrame(precision_recall, index = cols).transpose()
precision_recall_all = precision_recall_all.append(precision_recall)

print(results)

In [ ]:
# view results
results_df = pd.DataFrame(results)
results_df

In [ ]:
# print results and precision_recall to csv
# results_df.to_csv('results.csv') 
# precision_recall_all.to_csv('precision_recall.csv')

In [ ]:
precision_recall_all.head()

In [ ]:
# precision recall graph
sns.lineplot(data = precision_recall_all, x = 'recall', y = 'precision')



# CTU-13

In [ ]:
# list of scenarios
scenarios = ['1','2','3','4','5','6', '7','8','9','10', '11', '12', '13_test']
colab_path = '/content/'

# empty list to capture statistics for each scenario
results = []
precision_recall_all = pd.DataFrame()

# iterate over scenarios to build, train, and evaluate LSTM model
for filename in scenarios:

    # load the dataset
    scenario = pd.read_csv(colab_path + filename + '.txt', header=None)
    
    #impute missing vlaues
    imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean',verbose=0)
    imputer = imputer.fit(scenario.iloc[:, 1:])
    scenario.iloc[:, 1:] = imputer.transform(scenario.iloc[:, 1:])

    # split into input (X) and output (y) variables
    X = scenario.iloc[:,1:]
    y = scenario.iloc[:,0]

    # split into training (80%), validation (10%), and test (10%) sets
    rand_seed = 123
    X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size = 0.8,
                                                      random_state = rand_seed,
                                                      stratify = y)
    X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, 
                                                        train_size = 0.5,
                                                        random_state = rand_seed,
                                                        stratify = y_rem)
    
    #Creating and fitting model
    model = AutoEncoder(contamination=0.05, hidden_neurons=[3,3],
                        epochs=8, optimizer='Nadam')
    # model.fit(X_train)
    model.fit(X_train, y_train)
    
    #make predictions with model and print classification report
    predictions = (model.predict(X_test) > .5).astype(int)
    print(filename)
    print(classification_report(y_test, predictions))

    # calculate metrics for each model
    precision, recall, fscore, support = score(y_test, predictions, 
                                                   average = 'binary')
    try:
      auc = roc_auc_score(y_test, predictions)
    except ValueError:
      pass
    accuracy = accuracy_score(y_test, predictions)


    # append metrics to results
    results.append(
        {
            'Scenario': filename,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': fscore,
            'AUC': auc
        })

    # write data for precision recall curves to csv
    cols = ['precision', 'recall', 'thresholds']
    precision_recall = precision_recall_curve(y_test, model.predict(X_test))
    precision_recall = pd.DataFrame(precision_recall, index = cols).transpose()
    precision_recall['scenario'] = filename
    precision_recall_all = precision_recall_all.append(precision_recall)

    print("scenario", filename, "complete")

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_95 (Dense)            (None, 28)                812       
                                                                 
 dropout_76 (Dropout)        (None, 28)                0         
                                                                 
 dense_96 (Dense)            (None, 28)                812       
                                                                 
 dropout_77 (Dropout)        (None, 28)                0         
                                                                 
 dense_97 (Dense)            (None, 3)                 87        
                                                                 
 dropout_78 (Dropout)        (None, 3)                 0         
                                                                 
 dense_98 (Dense)            (None, 3)               

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Undefin

1202/1202 [==============================] - 2s 1ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
<ipython-input-13-e37046d952a9>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precision_recall_all = precision_recall_all.append(precision_recall)


...

<ipython-input-13-e37046d952a9>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precision_recall_all = precision_recall_all.append(precision_recall)


scenario 12 complete


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(


Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_155 (Dense)           (None, 28)                812       
                                                                 
 dropout_124 (Dropout)       (None, 28)                0         
                                                                 
 dense_156 (Dense)           (None, 28)                812       
                                                                 
 dropout_125 (Dropout)       (None, 28)                0         
                                                                 
 dense_157 (Dense)           (None, 3)                 87        
                                                                 
 dropout_126 (Dropout)       (None, 3)                 0         
                                                                 
 dense_158 (Dense)           (None, 3)               

<ipython-input-13-e37046d952a9>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precision_recall_all = precision_recall_all.append(precision_recall)


In [ ]:
results_df = pd.DataFrame(results).sort_values('Scenario')
results_df

In [ ]:
# print results and precision_recall to csv
# results_df.to_csv('results.csv') 
# precision_recall_all.to_csv('precision_recall.csv')

In [ ]:
# precision recall graphs for each scenario
graph = sns.FacetGrid(precision_recall_all, col = 'scenario', col_wrap = 3)
graph.map(sns.lineplot, 'recall', 'precision')